## Remote ID Sppoofing Attack

In [1]:
from config import Color
from helpers import clean
from helpers.coordinates import ENU, ENUPose, GRAPose
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)
from simulator.gazebo.config import GazWP
from simulator.QGroundControl.config import QGCWP

clean(reset_folders = [])

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

# east, north, up, heading
base_homes= ENUPose.list([(-10, 0, 0, 0),(0, -5, 0, 0)])
base_paths = [ENUPose.list([(0, 0, 5, 0),(20, 0, 5, 0)]),
              [ENUPose(0, 0, 5,0)]]
              
colors=[
    Color.GREEN,
    Color.RED,
]
msn_delays=[0,0]
speeds=[1.0,1.0]  # m/s
msn_land=[True,False]
radar_radius=10
safety_radius =5

## Visualizers

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)


true_pos = enu_origin.to_abs(base_homes[1]).to_abs(base_paths[1][0]).unpose()
fake_pos = enu_origin.to_abs(enu_origin).to_abs(ENU(x=0,y=0,z=5).pose()).unpose()

true_marker = GazWP(name="true_pos",
                    group="true_pos",
                    pos=true_pos,
                    color=Color.RED)
fake_marker = GazWP(name="fake_pos",
                group="fake_pos",
                pos=fake_pos,
                color=Color.ORANGE)

avoid_zone = GazWP(name="avoid_zone",
                group="avoid_zone",
                pos=fake_pos,
                color=Color.RED,
                radius=safety_radius,
                alpha=0.85)

radar_zone = GazWP(name="radar_zone",
                group="radar_zone",
                pos=true_pos,
                color=Color.ORANGE,
                radius=radar_radius,
                alpha=0.9)


gaz= Gazebo(gaz_config,gra_origin)
for mark in [true_marker,fake_marker,avoid_zone,radar_zone]:
    gaz.markers.append(mark)

gaz.show()

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)
hev_iter = zip(base_paths,base_homes,colors,msn_delays,msn_land,speeds)
for path,home,color,delay,land,speed in hev_iter:
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay,land=land,speed=speed)

qgc=QGC(qgc_config)
origin_qgc = QGCWP(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)

true_marker = QGCWP(name="true_pos",
                    pos=gra_origin.to_abs(true_pos).unpose(),
                    color=Color.RED)
fake_marker = QGCWP(name="fake_pos",
                pos=gra_origin.to_abs(fake_pos).unpose(),
                color=Color.ORANGE)
avoid_zone = QGCWP(name="avoid_zone",
                pos=gra_origin.to_abs(fake_pos).unpose(),
                color=Color.RED)

radar_zone = QGCWP(name="radar_zone",
                pos=gra_origin.to_abs(true_pos).unpose(),
                color=Color.ORANGE,
)


for mark in [true_marker,fake_marker,avoid_zone,radar_zone]:
    qgc.markers.append(mark)

qgc.show()

## No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

In [6]:
qgc_config.vehicles

[QGCVehicle(home=GRAPose(lat=np.float64(-35.36332799994992), lon=np.float64(149.16511406820254), alt=np.float64(7.831696711162024e-06), heading=0), mission=Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.3633279999499), lon=np.float64(149.1651140682887), alt=np.float64(5.0000078311989915)), color=green), QGCWP(name='wp_1', pos=GRA(lat=np.float64(-35.36332799994992), lon=np.float64(149.1653341317113), alt=np.float64(5.000007830390474)), color=green)], delay=0, land=True, speed=1.0)),
 QGCVehicle(home=GRAPose(lat=np.float64(-35.36337306646448), lon=np.float64(149.1652241), alt=np.float64(1.96678708108014e-06), heading=0), mission=Mission(traj=[QGCWP(name='wp_0', pos=GRA(lat=np.float64(-35.36337306642903), lon=np.float64(149.1652241), alt=np.float64(5.000001966938339)), color=red)], delay=0, land=False, speed=1.0))]

## Launch Simulation

In [7]:
simulator = Simulator(
    gra_origin=gra_origin,
	visualizers=[gaz],
	gcs_system_ids={f"{Color.GREEN.name}_{Color.GREEN.emoji}" :[1],
                f"{Color.RED.name}_{Color.RED.emoji}" : [255]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

16:22:31 - Oracle ⚪ - INFO - 🖥️  Gazebo launched for realistic simulation and 3D visualization.
16:22:32 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 2204339)
16:22:32 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 2204341)


## Oracle checking

In [8]:
orac.run()

16:22:32 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 2 vehicles and 2 GCSs
16:24:50 - Oracle ⚪ - INFO - Received DONE from GCS GREEN_🟩
16:24:50 - Oracle ⚪ - INFO - UAV 1 completed mission and exited


KeyboardInterrupt: 